<a href="https://colab.research.google.com/github/NoeliaDuranL/SIS420/blob/main/Examen_de_laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

%matplotlib inline

import pandas as pd


In [310]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [311]:
# Leer datos
df = pd.read_csv('/content/drive/MyDrive/SIS420/Practica Laboratorios/Lab 1/kc_house_data.csv', delimiter=',')

In [312]:
!ls
%mkdir data
!ls

data  drive  sample_data
mkdir: cannot create directory ‘data’: File exists
data  drive  sample_data


In [313]:
import sys
import shutil

# Procesamiento de datos

In [314]:
# Obtener la cantidad de datos no nulos en cada columna
column_counts_before = df.count()

# Imprimir la cantidad de datos en cada columna
print("Cantidad de datos en cada columna: ")
print(column_counts_before)

Cantidad de datos en cada columna: 
id               21613
date             21613
price            21613
bedrooms         21613
bathrooms        21613
sqft_living      21613
sqft_lot         21613
floors           21613
waterfront       21613
view             21613
condition        21613
grade            21613
sqft_above       21613
sqft_basement    21613
yr_built         21613
yr_renovated     21613
zipcode          21613
lat              21613
long             21613
sqft_living15    21613
sqft_lot15       21613
dtype: int64


In [315]:
# Seleccionar las columnas necesarias para X y y
selected_columns = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


# Crear un nuevo DataFrame con las columnas seleccionadas
df_selected = df[selected_columns]

# Mostrar las primeras filas del nuevo DataFrame
print(df_selected.head())

      price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  221900.0         3       1.00         1180      5650     1.0           0   
1  538000.0         3       2.25         2570      7242     2.0           0   
2  180000.0         2       1.00          770     10000     1.0           0   
3  604000.0         4       3.00         1960      5000     1.0           0   
4  510000.0         3       2.00         1680      8080     1.0           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0     0          3      7        1180              0      1955             0   
1     0          3      7        2170            400      1951          1991   
2     0          3      6         770              0      1933             0   
3     0          5      7        1050            910      1965             0   
4     0          3      8        1680              0      1987             0   

   zipcode      lat     long  sqft_living15 

In [316]:
data = df_selected

# Seleccion de datos

In [317]:
# columnas relevantes para X
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
X = data[features].values

# Phising' para y
y = data['price'].values

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
train_size = int(0.8 * len(data))
Xp, yp = X[train_size:], y[train_size:]
X, y = X[:train_size], y[:train_size]


m_train = len(y)

m = y.size

# Imprimir las formas de los datos
print(X.shape)
print(y.shape)
print(Xp.shape)
print(yp.shape)

# Imprimir las etiquetas de entrenamiento
print(y)

(17290, 18)
(17290,)
(4323, 18)
(4323,)
[ 221900.  538000.  180000. ...  775000. 2250000. 1350000.]


# Configuracion de capas

In [318]:
# Configurando las capas de
input_layer_size  = 18   # Entrada de 18 características, sin contar id
hidden_layer_size = 7   # 10 unidades ocultas(es un hiperparametro, controla la complejidad del modelo)
#output_layer_size
num_labels = 1    # Una sola salida (precio)

print("la cantidad de elementos en la capa de entrada es: ",input_layer_size)
print("la cantidad de elementos en la capa de oculta es: ",hidden_layer_size)
print("la cantidad de elementos en la capa de salida es: ",num_labels)



la cantidad de elementos en la capa de entrada es:  18
la cantidad de elementos en la capa de oculta es:  7
la cantidad de elementos en la capa de salida es:  1


# Inicializacion de parametros theta

In [319]:
# Inicializa los pesos en las variables Theta1 y Theta2
pesos = {}
pesos['Theta1'] = np.random.rand(hidden_layer_size , input_layer_size + 1) #+1 porque se incluye el sesgo
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size + 1) #+1 porque se incluye el sesgo

# Carga los pesos en las variables Theta1 y Theta2
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

# Con valores pequeños alatorios
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)


(133,)
(8,)


In [320]:
# Concatenación de thetas en un solo array
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(141,)


# Sigmoide y gradiente

In [321]:
def linear_activation(z):
    """Función de activación lineal."""
    return z

def linear_gradient(z):
    """Gradiente de la función de activación lineal (siempre es 1)."""
    return np.ones_like(z)

# Funcion de costo

In [322]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    """
    Calcula la función de costo y gradientes para una red neuronal de dos capas.

    Parámetros:
    nn_params : array_like
        Parámetros de la red neuronal desenrollados en un único vector.
    input_layer_size : int
        Tamaño de la capa de entrada.
    hidden_layer_size : int
        Tamaño de la capa oculta.
    num_labels : int
        Número de etiquetas (clases).
    X : array_like
        Datos de entrenamiento.
    y : array_like
        Etiquetas de entrenamiento.
    lambda_ : float, opcional
        Parámetro de regularización.

    Devuelve:
    J : float
        Valor de la función de costo.
    grad : array_like
        Gradientes de los parámetros de la red neuronal.
    """

    # Reshape nn_params para obtener Theta1 y Theta2
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size  # Número de ejemplos de entrenamiento

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    # Propagación hacia adelante
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)  # Agregar sesgo a los datos de entrada
    z2 = a1.dot(Theta1.T)  # Calcular la salida de la capa oculta
    a2 = linear_activation(z2)  # Aplicar función de activación a la salida de la capa oculta
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)  # Agregar sesgo a la salida de la capa oculta
    z3 = a2.dot(Theta2.T)  # Calcular la salida de la capa de salida
    a3 = linear_activation(z3)  # Aplicar función de activación a la salida de la capa de salida

    # Asegurar que y tenga la forma adecuada
    y = y.reshape(-1, 1)

    # Función de costo
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))
    J = (1 / (2 * m)) * np.sum(np.square(a3 - y))
    J += reg_term

    # Retropropagación=propagar el error desde la capa de salida hasta la capa de entrada de la red
    delta_3 = a3 - y  # Calcular el error en la capa de salida
    delta_2 = delta_3.dot(Theta2)[:, 1:] * linear_gradient(z2)  # Calcular el error en la capa oculta

    Delta1 = delta_2.T.dot(a1)  # Calcular la acumulación de gradientes para Theta1
    Delta2 = delta_3.T.dot(a2)  # Calcular la acumulación de gradientes para Theta2

    # Regularización de gradientes
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] += (lambda_ / m) * Theta1[:, 1:]
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] += (lambda_ / m) * Theta2[:, 1:]

    # Desenrollar gradientes
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad


# Normalizacion

In [323]:
def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array x
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #promedio de cada columna
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)

    sigma[sigma == 0] = 1

    #normalizacion
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [324]:
X_norm, mu, sigma = featureNormalize(X)

In [325]:
print("Longitud de nn_params:", len(nn_params))
print("Expected size for Theta2:", (hidden_layer_size + 1) * num_labels)

Longitud de nn_params: 141
Expected size for Theta2: 8


# Calculo de costo(lambda)

In [326]:
#calculo costo
#lambda=10= 210668998573.230896
#lambda=100= 210669008419.584686
#lambda=1000=210669106883.122620
#lambda=10000=210670091518.501831

lambda_ = 100000
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X_norm, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
print('El costo debe esta cercano a               : ')

Costo en parametros (cargado de ex4weights): 209122492949.250305 
El costo debe esta cercano a               : 


# Inicialización de los pesos de la una capa

In [327]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))  # Inicializar la matriz de pesos con ceros
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init  # Inicializar aleatoriamente los pesos en el rango (-epsilon_init, epsilon_init)

    return W


In [328]:
print('Inicialización de parámetros de redes neuronales...')

# Inicializar pesos de la capa oculta y de salida
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollar parámetros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)


Inicialización de parámetros de redes neuronales...


# Entrenamiento

In [329]:
#entrenamiento con el uso de la funcion optimize.minimize
# Después de completar la tarea, cambia el valor de maxiter a uno más grande
options= {'maxiter': 10000} #iteracciones

#
lambda_ = 100

# Crear una versión abreviada de la función de costo a minimizar
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Ahora, costFunction es una función que toma solo un argumento
# (los parámetros de la red neuronal)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# Obtener la solución de la optimización
nn_params = res.x #almacenan

# Obtener Theta1 y Theta2 a partir de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))


<ipython-input-329-641e3251a7eb>:15: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,


# Parametros theta1 y theta2

In [330]:
#parametros de matriz theta1, pesos de la neuora en la capa oculta, son 10
for i in range(len(Theta1)):
    print("Thetha",i,"es: ", Theta1[i])
    print("")

Thetha 0 es:  [-3.46480147e-02  2.79236613e+01 -5.03499379e+01 -3.44228542e+00
  5.28804158e-01 -5.86968195e+00 -1.33447056e+02 -5.14489144e+01
 -2.59225646e+01 -8.78885951e+01 -6.28253468e-01  7.50631555e+00
 -9.74265725e+00  1.12740922e+00  6.54576824e-03 -2.67700525e+02
  2.68420843e+01 -2.09495145e+00 -9.32643956e-02]

Thetha 1 es:  [-2.27045165e-02  3.91515857e+01 -1.92263243e+01  7.50846042e+00
 -4.91716596e-01 -1.03759331e+01 -2.39339846e+01 -6.43908760e+01
 -1.30048259e+01 -7.95438759e+01  1.08445004e+01  1.40790033e+01
 -3.47830203e+00  6.47235183e-01 -8.36169729e-01 -4.92949001e+01
  4.91797102e+00  1.10608543e+01 -1.21792146e+00]

Thetha 2 es:  [-1.80644815e-01 -5.73553189e+01  3.31988917e+01 -8.30437007e+00
 -1.36458680e+00  1.31169404e+01  4.17698176e+01  7.98700614e+01
  1.98892147e+01  1.23495777e+02 -1.30104036e+01 -1.04648813e+01
  4.12736498e-01  1.21487519e-01 -2.43620181e+00  7.37991156e+01
 -9.03037359e+00 -9.82411418e+00  5.72960872e-01]

Thetha 3 es:  [ 1.9944160

In [331]:
#parametros de matriz theta2, pesos de la capa de salida
for i in range(len(Theta2)):
    print("Thetha",i,"es: ", Theta1[i])
    print("")

Thetha 0 es:  [-3.46480147e-02  2.79236613e+01 -5.03499379e+01 -3.44228542e+00
  5.28804158e-01 -5.86968195e+00 -1.33447056e+02 -5.14489144e+01
 -2.59225646e+01 -8.78885951e+01 -6.28253468e-01  7.50631555e+00
 -9.74265725e+00  1.12740922e+00  6.54576824e-03 -2.67700525e+02
  2.68420843e+01 -2.09495145e+00 -9.32643956e-02]



In [332]:
def predict(Theta1, Theta2, X): #calculo de etiquetas predichas para las muestras de x utilizando el entrenamiento de t1 y t2

   # Calcular la salida utilizando los pesos entrenados
    h1 = np.dot(np.concatenate([np.ones((X.shape[0], 1)), X], axis=1), Theta1.T)  # Salida de la capa oculta
    output = np.dot(np.concatenate([np.ones((h1.shape[0], 1)), h1], axis=1), Theta2.T)  # Salida final

    return output

# Predicciones

In [333]:
#Predecir los precios de ejemplos:
matriz_datos = np.array([
    [3, 1, 1180, 5650, 1, 0, 0, 3, 7, 1180, 0, 1955, 0, 98178, 47.5112, -122.257, 1340, 5660],
    [3, 2.5, 2570, 7242, 2, 0, 0, 1, 8, 1500, 0, 1960, 0, 98125, 47.5141, -122.250, 1500, 6000],
    [3, 2, 1600, 7000, 1, 0, 0, 1, 6, 1600, 0, 1980, 0, 98003, 47.5201, -122.233, 1600, 7000],
    [5, 3, 2000, 8000, 2, 0, 1, 0, 9, 2000, 0, 1970, 0, 98001, 47.5251, -122.222, 2000, 8000],
    [2, 1, 1000, 5000, 1, 0, 0, 0, 5, 1000, 0, 2000, 0, 98002, 47.5301, -122.211, 1000, 5000]

])


#Normalizacion
matriz_datos= (matriz_datos - mu) / sigma


#Y predicha
pred = predict(Theta1, Theta2, matriz_datos)

print("Predicciones:")
i = 0
for i in range(len(matriz_datos)):
    print("Precio",i+1,"es:", pred[i])
    print("")




Predicciones:
Precio 1 es: [-227352.72360978]

Precio 2 es: [-106395.45090198]

Precio 3 es: [-329778.75570404]

Precio 4 es: [-11063.75636864]

Precio 5 es: [-474772.91211601]



In [334]:
#Creamos la funcion:
def mean_squared_error(y_pred, y_actual):
     # Calcular la diferencia entre las etiquetas predichas y reales
    resta = y_pred - y_actual
    # Calcular el error cuadrático
    err_cuadrado = np.sum(resta ** 2)
    # Calcular el error cuadrático medio
    mse = err_cuadrado / len(y_pred)

    return mse

In [335]:
# Normalizar los datos de prueba
Xp = (Xp - mu) / sigma

# Realizar predicciones con la red neuronal
pred = predict(Theta1, Theta2, Xp)

# Imprimir las predicciones
print(pred)


[[190278.22614846]
 [190278.22614846]
 [-14566.66071894]
 ...
 [-79218.46832813]
 [-24387.44442868]
 [-79511.85357319]]


In [336]:
mse = mean_squared_error(pred, yp)
#calculamos el error cuadratico medio:
print('Error Cuadrático Medio (MSE) = ' + str(mse))
print('Raíz del Error Cuadrático Medio (RMSE) = ' + str(np.sqrt(mse)))

Error Cuadrático Medio (MSE) = 2298365610812348.0
Raíz del Error Cuadrático Medio (RMSE) = 47941272.51974386
